In [1]:
import json
import bz2
import pandas

features = pandas.read_csv('./final_test/features.csv', index_col='match_id')

match_result_col_names=["duration", "radiant_win", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"]
features.drop(match_result_col_names, inplace=True, axis=1)

features_count = features.count()
col_length=max(features_count)
print list(features_count[features_count<col_length].index)


SyntaxError: invalid syntax (<ipython-input-1-b81c5ae5a470>, line 12)

Обоснование пропусков:
    first_blood_time               событие "первая кровь" не успело произойти за первые 5 минут
    first_blood_team               событие "первая кровь" не успело произойти за первые 5 минут
    first_blood_player1            событие "первая кровь" не успело произойти за первые 5 минут
    first_blood_player2            событие "первая кровь" не успело произойти за первые 5 минут
    radiant_bottle_time            "radiant" команда не приобрела предмета "bottle" за первые 5 минут
    radiant_courier_time           "radiant" команда не приобрела предмета "courier" за первые 5 минут
    radiant_flying_courier_time    "radiant" команда не приобрела предмета "flying_courier" за первые 5 минут
    radiant_first_ward_time        "radiant" команда не установила "наблюдателя" за первые 5 минут
    dire_bottle_time               "dire" команда не приобрела предмета "bottle" за первые 5 минут
    dire_courier_time              "dire" команда не приобрела предмета "countier" за первые 5 минут
    dire_flying_courier_time       "dire" команда не приобрела предмета "flying_countier" за первые 5 минут
    dire_first_ward_time           "dire" команда не установила "наблюдателя" за первые 5 минут

# Подход 1: градиентный бустинг "в лоб"

In [98]:
import json
import bz2
import pandas
import time
import datetime
import numpy as np
from sklearn.model_selection import KFold
#from sklearn.model_selection import cross_val_score

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

features = pandas.read_csv('./final_test/features.csv', index_col='match_id')

match_result_col_names=["duration", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"]
features.drop(match_result_col_names, inplace=True, axis=1)
features=features.fillna(0)

features_count = features.count()
col_length=max(features_count)
if len(features_count[features_count<col_length].index)>0:
    print "There is NA values" 

y=features["radiant_win"]
features.drop("radiant_win", inplace=True, axis=1)
X=features
tree_count=[10, 20, 30, 50, 100, 150, 200, 250]
n_folds=5

my_cv = KFold(n_folds, shuffle=True, random_state=42)
for i in tree_count:
    print "tree count: "+str(i)
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=i, random_state=241)
    score=cross_val_score(clf, X, y, cv=my_cv, scoring='roc_auc').mean()
    print 'roc_auc_score', score
    print 'Time elapsed:', datetime.datetime.now() - start_time
    

tree count: 10
roc_auc_score 0.664850687975
Time elapsed: 0:01:22.636535
tree count: 20
roc_auc_score 0.682461876804
Time elapsed: 0:02:16.470034
tree count: 30
roc_auc_score 0.690006471039
Time elapsed: 0:03:28.726259
tree count: 50
roc_auc_score 0.697494360947
Time elapsed: 0:05:15.145285
tree count: 100
roc_auc_score 0.706222978399
Time elapsed: 0:10:40.055286
tree count: 150
roc_auc_score 0.710470757444
Time elapsed: 0:15:48.237984
tree count: 200
roc_auc_score 0.71368575208
Time elapsed: 0:20:50.793628
tree count: 250
roc_auc_score 0.715909455738
Time elapsed: 0:26:20.661543


# Подход 2: логистическая регрессия

In [14]:
import json
import bz2
import pandas
import time
import datetime
import numpy as np
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

features = pandas.read_csv('./final_test/features.csv', index_col='match_id')

match_result_col_names=["duration", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"]
features.drop(match_result_col_names, inplace=True, axis=1)
features=features.fillna(0)
y=features["radiant_win"]
features.drop("radiant_win", inplace=True, axis=1)
X=features

categor_col=['lobby_type', 'r{1..5}_hero', 'd{1..5}_hero']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
eps=10**-6
k=0.1
n_folds=5
C_pow_range = range(-5, 6)
C = [10.0 ** i for i in C_pow_range]

my_cv = KFold(n_folds, shuffle=True, random_state=42)
for i in C:
    print "C: "+str(i)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=i, random_state=42, penalty='l2')
    score=cross_val_score(clf, X_scaled, y, cv=my_cv, scoring='roc_auc').mean()
    print 'roc_auc_score', score
    print 'Time elapsed:', datetime.datetime.now() - start_time
       
non_hero_features = ['lobby_type']
hero_features = list(features.filter(regex=(".._hero")))
features.drop(non_hero_features, inplace=True, axis=1)
features.drop(hero_features, inplace=True, axis=1)
X=features
X_scaled_hero = scaler.fit_transform(X)

print ""
print "without hero features"
for i in C:
    print "C: "+str(i)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=i, random_state=42, penalty='l2')
    score=cross_val_score(clf, X_scaled_hero, y, cv=my_cv, scoring='roc_auc').mean()
    print 'roc_auc_score', score
    print 'Time elapsed:', datetime.datetime.now() - start_time


C: 1e-05
roc_auc_score 0.695161685464
Time elapsed: 0:00:05.076896
C: 0.0001
roc_auc_score 0.711357747143
Time elapsed: 0:00:06.693831
C: 0.001
roc_auc_score 0.716363537821
Time elapsed: 0:00:11.819545
C: 0.01
roc_auc_score 0.716550269726
Time elapsed: 0:00:13.523492
C: 0.1
roc_auc_score 0.716527148666
Time elapsed: 0:00:15.442959
C: 1.0
roc_auc_score 0.716522600363
Time elapsed: 0:00:15.220127
C: 10.0
roc_auc_score 0.716522288896
Time elapsed: 0:00:14.848891
C: 100.0
roc_auc_score 0.716522295258
Time elapsed: 0:00:14.966803
C: 1000.0
roc_auc_score 0.716522322796
Time elapsed: 0:00:14.466040
C: 10000.0
roc_auc_score 0.716522327033
Time elapsed: 0:00:14.571935
C: 100000.0
roc_auc_score 0.716522331269
Time elapsed: 0:00:14.923204
<class 'pandas.core.frame.DataFrame'>
without hero features
C: 1e-05
roc_auc_score 0.695100191344
Time elapsed: 0:00:03.833406
C: 0.0001
roc_auc_score 0.711339145863
Time elapsed: 0:00:05.543867
C: 0.001
roc_auc_score 0.716375799908
Time elapsed: 0:00:09.740482


# Подход 2: логистическая регрессия. Метод "мешка слов" по героям

In [21]:
import json
import bz2
import pandas as pd
import time
import datetime
import numpy as np
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

features = pd.read_csv('./final_test/features.csv', index_col='match_id')
match_result_col_names=["duration", "tower_status_radiant", "tower_status_dire", "barracks_status_radiant", "barracks_status_dire"]
features.drop(match_result_col_names, inplace=True, axis=1)
features=features.fillna(0)
y=features["radiant_win"]
features.drop("radiant_win", inplace=True, axis=1)

hero_features = list(features.filter(regex=(".._hero")))
hero_number, hero_count = np.unique(features.filter(regex=(".._hero")), return_index=True)
N=len(hero_number)
print "count of unique heros: " + str(N)
X_pick = pd.DataFrame(index=features.index, columns=['hero_%d' % i for i in range(N)])
X_pick = X_pick.fillna(0)
#X_pick = np.zeros((features.shape[0], N))
## 5 - count of heros from one side
for i, match_id in enumerate(features.index):
    for p in xrange(5):
        r_hero=features.ix[match_id, 'r%d_hero' % (p+1)]
        d_hero=features.ix[match_id, 'd%d_hero' % (p+1)]
        X_pick.iat[i, list(hero_number).index(r_hero)] = 1
        X_pick.iat[i, list(hero_number).index(d_hero)] = -1

non_hero_features = ['lobby_type']
#features.drop(non_hero_features, inplace=True, axis=1)
features.drop(hero_features, inplace=True, axis=1)
X=pd.concat([features, X_pick], axis=1)
scaler = StandardScaler()
X_scaled_hero = scaler.fit_transform(X)

eps=10**-6
k=0.1
n_folds=5
C_pow_range = range(-5, 6)
C = [10.0 ** i for i in C_pow_range]

for i in C:
    print "C: "+str(i)
    start_time = datetime.datetime.now()
    clf = LogisticRegression(C=i, random_state=42, penalty='l2')
    score=cross_val_score(clf, X_scaled_hero, y, cv=my_cv, scoring='roc_auc').mean()
    print 'roc_auc_score', score
    print 'Time elapsed:', datetime.datetime.now() - start_time

count of unique heros: 108
C: 1e-05
roc_auc_score 0.714803920935
Time elapsed: 0:00:07.330859
C: 0.0001
roc_auc_score 0.742729008456
Time elapsed: 0:00:10.391862
C: 0.001
roc_auc_score 0.751598744515
Time elapsed: 0:00:19.108822
C: 0.01
roc_auc_score 0.751953651886
Time elapsed: 0:00:25.123754
C: 0.1
roc_auc_score 0.75192047777
Time elapsed: 0:00:28.435210
C: 1.0
roc_auc_score 0.751914448853
Time elapsed: 0:00:26.498407
C: 10.0
roc_auc_score 0.751914484873
Time elapsed: 0:00:25.805119
C: 100.0
roc_auc_score 0.751914457324
Time elapsed: 0:00:28.390648
C: 1000.0
roc_auc_score 0.751914463677
Time elapsed: 0:00:28.432963
C: 10000.0
roc_auc_score 0.75191445944
Time elapsed: 0:00:26.606148
C: 100000.0
roc_auc_score 0.75191445944
Time elapsed: 0:00:28.420706
